![CH5-ADS.png](.\Media\CH5-ADS.png)
# <span style="color:#cc5500;">Useful Undocumented DBCC Commands</span>

Microsoft uses Undocumented Commands for internal testing and enhancing the quality of our products.  We generally don't blog about them because they can introduce risk to your production environment.  I am providing informations and demo's to help you practice and master your skills in responding to corruptions.  I strongly caution you to do your practice on a test/dev server and not in production.

## <span style="color:#cc5500;">DBCC IND</span>

- DBCC IND dumps a tabular list of all pages of all page types that are allocated to a table and all of its' indexes
    - dbccind( { 'dbname' | dbid}, { 'objname' | objid}, { nonclustered indid| 1 | 0 | -1 | -2 } \[, partition\_number\] )
- Third parameter options:
    - Nonclustered index ID –just that nonclustered index
    - 0 – just the heap or clustered index
    - 1 – just the clustered index
    - \-1 – the heap or clustered index and all nonclustered indexes
    - \-2 – IAM pages from the heap or clustered index and all nonclustered indexes
- Works in all versions of SQL Server

## <span style="color:#cc5500;">DBCC PAGE</span>

- DBCC PAGE dumps an individual data page
    - dbcc page ( {'dbname' | dbid}, filenum, pagenum\[, printopt={0|1|2|3} \])
    - Use WITH TABLERESULTS to get tabular output
- Requires trace flag 3604 to be enabled to get results
- Four parts to the output:
    - The BUF structure from the buffer pool
    - The page header
    - The allocation status of the page and the extent it is part of
    - Dump of the page contents
- Printopt options:
    - 0 – everything except the page contents
    - 1 – hex dump of records on the page
    - 2 – hex dump of the whole page
    - 3 – hex dump and interpretation of records on the page

## <span style="color:#cc5500;">DBCC IND and DBCC PAGE Demo</span>

### <span style="color:rgb(0, 204, 153);">Run the Code block below</span>

1. Click the run icon below
2. If ADS prompts you for a connection, enter the correct SQL Server and authentication account
3. View the results of the query by scrolling down to the results set

In [ ]:
-- Create a test database for this demo
USE [master];
GO

IF DATABASEPROPERTYEX (N'Company', N'Version') > 0
BEGIN
	ALTER DATABASE [Company] SET SINGLE_USER
		WITH ROLLBACK IMMEDIATE;
	DROP DATABASE [Company];
END
GO

-- Create the database
CREATE DATABASE [Company] ON PRIMARY (
    NAME = N'Company',
    FILENAME = N'C:\Program Files\Microsoft SQL Server\MSSQL14.VIPER\MSSQL\DATA\Company.mdf')  -- Modify path for your environment
LOG ON (
    NAME = N'Company_log',
    FILENAME = N'C:\Program Files\Microsoft SQL Server\MSSQL14.VIPER\MSSQL\DATA\Company_log.ldf');  -- Modify path for your environment
GO

In [ ]:
-- Create a table and some indexes so that we have data structures to look at
USE Company
GO
CREATE TABLE [RandomData] (
	[c1]  INT IDENTITY,
	[c2]  DATETIME2,
	[c3]  VARCHAR (5000) DEFAULT REPLICATE ('a', 5000),
	[c4]  VARCHAR (5000) DEFAULT REPLICATE ('b', 5000),
	[c5]  VARCHAR (MAX)  DEFAULT REPLICATE ('c', 5000));
GO

INSERT INTO [RandomData] DEFAULT VALUES;
GO 10

CREATE CLUSTERED INDEX [RandomData_CL_c1]
ON [RandomData] ([c1]);

CREATE NONCLUSTERED INDEX [RandomData_NCL_c2]
ON [RandomData] ([c2]);
GO

In [ ]:
-- DBCC IND dump styles
-- Notice the allocation unit types and page linkages

-- dump style 0 = just the heap or clustered index
DBCC IND (N'Company', N'RandomData', 0);
GO
-- dump style 1 = just the clustered index
DBCC IND (N'Company', N'RandomData', 1);
GO
-- dump style 2 = the pages just for index id 2
DBCC IND (N'Company', N'RandomData', 2);
GO
-- dump style -1 = the heap or clustered index and all non clustered indexes.  All pages for all objects.
DBCC IND (N'Company', N'RandomData', -1);
GO
-- dump style -2 IAM pages from the heap or clustered indexes
DBCC IND (N'Company', N'RandomData', -2);
GO

In [ ]:
-- To use DBCC PAGE we need to turn on trace flag 3604.  Without it, DBCC PAGE cannot output results to your connection.
DBCC TRACEON (3604);
GO
-- Run DBCC IND to pick a random data page.  You are looking for  page  type 1
DBCC IND (N'Company', N'RandomData', 1);
GO

![CH5-DBCC-IND.jpg](.\Media\CH5-DBCC-IND.jpg) 

We want page type 1 for in-row data and I am going to pick a random page in the pic above, and I will use PagePFID 456 for my DBCC PAGE commands below

In [ ]:
-- Replace the XX with your value
-- print option 0 = everything but the page contents
DBCC TRACEON (3604) 
GO
DBCC PAGE (N'Company', 1, xx, 0);


In [ ]:
-- Replace the XX with your value
-- print option 1 = hex dump of records on the page
DBCC TRACEON (3604) 
GO
DBCC PAGE (N'Company', 1, xx, 1);

Feel free to practice with different dump style and different page types.  Dump style 3 is the most comprehensive